<a href="https://colab.research.google.com/github/rafaellabastos/savvyfix-ia-notebook/blob/rm552425/precificacao_dinamica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SavvyFix - Precificação dinâmica**

<p>2TDSPF</p>

<p>Douglas Magalhães de Araujo - 552008</p>
<p>Gustavo Argüello Bertacci - 551304</p>
<p>Luiz Fillipe Farias - 99519</p>
<p>Rafaella Monique do Carmo Bastos - 552425</p>

# **Dataset**

<p>Fonte: https://www.kaggle.com/code/jeffersonvalandro/analisando-inflacao-brasil/input?select=relatorio_preco_medio_mensal_2023.csv</p>

## Descrição do conjunto de dados
<p>O objetivo desse notebook é criar um modelo de Machine Learning utilizando a técnica de Regressão Linear para demonstrar os prováveis preços dinamizados ao longo dos meses com base nos meses anteriores.</p>
<p>Os dados foram coletados a partir da inflação dos produtos ao longo dos meses do ano de 2023.</p>
<p>Observação: com o tempo, este modelo será alimentado com dados a partir da localização, clima, demanda, procura e horário.</p>

## **Importação das bibliotecas**

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

## **Inicialização do Data Frame**

In [47]:
dados = pd.read_csv('relatorio_preco_medio_mensal_2023.csv')
dados.head()

,Produto/Unidade,Nível de Comercialização,U.F.,01/2023,02/2023,03/2023,04/2023,05/2023,06/2023,07/2023,08/2023,09/2023,10/2023,11/2023,12/2023
0,ABACATE (kg),ATACADO,RR,"6,83","6,67","6,83","7,00","7,00","6,00","6,00","6,60","7,00","7,00","7,00","7,00"
1,ABACAXI HAVAÍ (kg),PRODUTOR,PR,"1,85","2,31","2,30","2,30","2,30",NaN,NaN,NaN,NaN,NaN,"2,60","2,60"
2,ABACAXI (kg),ATACADO,RR,"2,59","2,65","2,79","6,00","8,00","4,50","4,50","4,95","5,00","7,50","7,00","7,50"
3,ABACAXI PÉROLA (6.50 kg),PRODUTOR,DF,"21,80","20,14","23,45","26,65","34,82","36,87","25,47","27,36","28,48","31,92","25,05","25,75"
4,ABACAXI PÉROLA (kg),NaN,AM,"3,81","3,75","3,75","3,90","4,00","4,25","4,02","3,94","4,27","4,25","4,51","4,23"


## **Visualização e tratamento do DataSet**

---



In [48]:
# Visualização das últimas linhas
dados.tail()

,Produto/Unidade,Nível de Comercialização,U.F.,01/2023,02/2023,03/2023,04/2023,05/2023,06/2023,07/2023,08/2023,09/2023,10/2023,11/2023,12/2023
2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2026,Fonte: Companhia Nacional de Abastecimento - C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2027,"As informações podem ser reproduzidas, desde q...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2028,Proibida a comercialização das informações dis...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# Quantidade de linhas e colunas do DataFrame
dados.shape

(2029, 15)

In [50]:
# Nomes das colunas
dados.columns

Index(['Produto/Unidade', 'Nível de Comercialização', 'U.F.', '01/2023',
       '02/2023', '03/2023', '04/2023', '05/2023', '06/2023', '07/2023',
       '08/2023', '09/2023', '10/2023', '11/2023', '12/2023'],
      dtype='object')

In [51]:
# Informação do formato de dados das colunas
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Produto/Unidade           551 non-null    object
 1   Nível de Comercialização  309 non-null    object
 2   U.F.                      2024 non-null   object
 3   01/2023                   1901 non-null   object
 4   02/2023                   1902 non-null   object
 5   03/2023                   1874 non-null   object
 6   04/2023                   1874 non-null   object
 7   05/2023                   1884 non-null   object
 8   06/2023                   1879 non-null   object
 9   07/2023                   1852 non-null   object
 10  08/2023                   1781 non-null   object
 11  09/2023                   1765 non-null   object
 12  10/2023                   1706 non-null   object
 13  11/2023                   1603 non-null   object
 14  12/2023                 

In [52]:
# Exclusão de colunas sem importância
dados = dados.drop('Nível de Comercialização', axis=1)
dados = dados.drop('U.F.', axis=1)
dados.head()

,Produto/Unidade,01/2023,02/2023,03/2023,04/2023,05/2023,06/2023,07/2023,08/2023,09/2023,10/2023,11/2023,12/2023
0,ABACATE (kg),"6,83","6,67","6,83","7,00","7,00","6,00","6,00","6,60","7,00","7,00","7,00","7,00"
1,ABACAXI HAVAÍ (kg),"1,85","2,31","2,30","2,30","2,30",NaN,NaN,NaN,NaN,NaN,"2,60","2,60"
2,ABACAXI (kg),"2,59","2,65","2,79","6,00","8,00","4,50","4,50","4,95","5,00","7,50","7,00","7,50"
3,ABACAXI PÉROLA (6.50 kg),"21,80","20,14","23,45","26,65","34,82","36,87","25,47","27,36","28,48","31,92","25,05","25,75"
4,ABACAXI PÉROLA (kg),"3,81","3,75","3,75","3,90","4,00","4,25","4,02","3,94","4,27","4,25","4,51","4,23"


In [54]:
# Duplicação da coluna Produto/Unidade
dados['Produto/Unidade_2'] = dados['Produto/Unidade']
dados.head()

,Produto/Unidade,01/2023,02/2023,03/2023,04/2023,05/2023,06/2023,07/2023,08/2023,09/2023,10/2023,11/2023,12/2023,Produto/Unidade_2
0,ABACATE (kg),"6,83","6,67","6,83","7,00","7,00","6,00","6,00","6,60","7,00","7,00","7,00","7,00",ABACATE (kg)
1,ABACAXI HAVAÍ (kg),"1,85","2,31","2,30","2,30","2,30",NaN,NaN,NaN,NaN,NaN,"2,60","2,60",ABACAXI HAVAÍ (kg)
2,ABACAXI (kg),"2,59","2,65","2,79","6,00","8,00","4,50","4,50","4,95","5,00","7,50","7,00","7,50",ABACAXI (kg)
3,ABACAXI PÉROLA (6.50 kg),"21,80","20,14","23,45","26,65","34,82","36,87","25,47","27,36","28,48","31,92","25,05","25,75",ABACAXI PÉROLA (6.50 kg)
4,ABACAXI PÉROLA (kg),"3,81","3,75","3,75","3,90","4,00","4,25","4,02","3,94","4,27","4,25","4,51","4,23",ABACAXI PÉROLA (kg)


In [57]:
# Conversão de uma coluna de dados categóricos para dados numéricos
le = LabelEncoder()

for column in dados.columns:
  if column!= 'Produto/Unidade' and dados[column].dtype == 'object':
    dados[column] = le.fit_transform(dados[column])

dados.head()

,Produto/Unidade,01/2023,02/2023,03/2023,04/2023,05/2023,06/2023,07/2023,08/2023,09/2023,10/2023,11/2023,12/2023,Produto/Unidade_2
0,ABACATE (kg),1120,1137,1118,1135,1214,1099,1068,1090,1177,1140,1109,1055,0
1,ABACAXI HAVAÍ (kg),43,505,479,447,473,1461,1420,1405,1386,1317,468,436,2
2,ABACAXI (kg),520,527,511,1039,1312,881,848,881,938,1167,1109,1083,1
3,ABACAXI PÉROLA (6.50 kg),570,557,591,608,805,825,635,657,654,710,559,526,3
4,ABACAXI PÉROLA (kg),771,775,752,715,849,862,815,735,818,770,768,723,4


In [58]:
# Informação do formato de dados das colunas pós-transformação
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Produto/Unidade    551 non-null    object
 1   01/2023            2029 non-null   int64 
 2   02/2023            2029 non-null   int64 
 3   03/2023            2029 non-null   int64 
 4   04/2023            2029 non-null   int64 
 5   05/2023            2029 non-null   int64 
 6   06/2023            2029 non-null   int64 
 7   07/2023            2029 non-null   int64 
 8   08/2023            2029 non-null   int64 
 9   09/2023            2029 non-null   int64 
 10  10/2023            2029 non-null   int64 
 11  11/2023            2029 non-null   int64 
 12  12/2023            2029 non-null   int64 
 13  Produto/Unidade_2  2029 non-null   int64 
dtypes: int64(13), object(1)
memory usage: 222.0+ KB


In [59]:
# Exclusão de linhas com valores Nan
dados = dados.dropna()
dados = dados.iloc[:-3]

In [60]:
# Visualização do DataFrame sem as linhas com NaN
dados.tail()

,Produto/Unidade,01/2023,02/2023,03/2023,04/2023,05/2023,06/2023,07/2023,08/2023,09/2023,10/2023,11/2023,12/2023,Produto/Unidade_2
2011,UVA INDÚSTRIA NIÁGARA (kg),894,883,1475,1426,997,999,994,990,1002,1041,1046,979,538
2012,UVA MOSCATO (kg),486,495,463,436,40,44,41,40,42,37,44,38,540
2013,UVA NIÁGARA (kg),28,29,20,24,19,20,21,18,17,16,20,11,541
2014,VACA GORDA (@),636,606,541,507,445,381,404,1405,1386,1317,1276,1249,543
2015,VACA GORDA (15 kg),692,692,619,532,578,435,410,1405,1386,1317,1276,1249,542


## **Separação dos dados em features e labels**

In [67]:
# Seleção de features (entradas)
remocao = ['Produto/Unidade', 'Produto/Unidade_2']
X = dados.drop(remocao, axis=1).values
X

array([[1120, 1137, 1118, ..., 1140, 1109, 1055],
       [  43,  505,  479, ..., 1317,  468,  436],
       [ 520,  527,  511, ..., 1167, 1109, 1083],
       ...,
       [  28,   29,   20, ...,   16,   20,   11],
       [ 636,  606,  541, ..., 1317, 1276, 1249],
       [ 692,  692,  619, ..., 1317, 1276, 1249]])

In [68]:
# Seleção da coluna target
y = dados['Produto/Unidade_2'].values
y

array([  0,   2,   1,   3,   4,   5,   6,   7,   9,   8,  63,  63,  10,
        65,  64,  67,  66,  69,  70,  68,  71,  72,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  27,  28,  29,  30,
        31,  24,  25,  26,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        42,  41,  44,  43,  46,  45,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  74,  75,  73,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  91,  92,  93,  95,  96,
        94,  86,  87,  89,  88,  90,  97,  98,  99, 101, 100, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 114, 115, 116, 113,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129, 128,
       130, 131, 132, 133, 134, 168, 169, 170, 135, 136, 137, 138, 139,
       140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
       153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
       166, 167, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

## **Separação dos dados para treinamento e teste**

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=27)

In [70]:
X_train.shape

(438, 12)

In [71]:
X_test.shape

(110, 12)

## **Treino do modelo de regressão linear**

In [72]:
# Treinar modelo
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()